# Stock predication

Анализ ценных бумаг преследует цель определить инвестиционное качество той или иной ценной бумаги и ее цены.

В нашем случае, это еще и предсказание цены на основе прошлых данных.


Анализ ценных бумаг преследует две цели:
Во-первых, он позволяет так представить важные факты, касающиеся акций или других ценных бумаг, чтобы это было удобно текущему или потенциальному владельцу этих активов.
Во-вторых, он позволяет сделать основанное на фактах и прикладных стандартах заключение о безопасности и привлекательности данных активов при текущей или предполагаемой цене.

# Import libraries and packages

In [68]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def preparation(historical_prices):
    try:
        historical_prices.rename(columns={"Date":"date","Open":"open","High":"high","Low":"low","Close":"close", "Adj Close":"adj close", "Price":"close", "Volume":"volume", "Vol.":"volume", "volume":"volume"}, inplace=True)
        # Drop na value
        historical_prices.dropna(inplace=True)
        if historical_prices.isna().any().any():
            raise ValueError("NaN values detected in the DataFrame.")
        # Convert date
        historical_prices['date'] = pd.to_datetime(historical_prices['date'], format="%m/%d/%Y")
        # Sorting dataset
        historical_prices.sort_values(by='date', inplace=True)
        return historical_prices
    except Exception as e:
        return f"An error occurred: {str(e)}"


historical_prices = preparation(pd.read_csv("./app/content/1288 Historical Data.csv"))



# **EDA - Exploratory Data Analysis (Исследовательский анализ данных)**

Попробуем ответить на следующие вопросы:
1.  Какая динамика изменения максимальной и минимальной цены  между месяцами за последний год?
2.  Какой день был самый волантильный за последний год?
3.  Какой тренд (рост или падание) за последние время у акции?
4.  Когда были смены тренда?
В каждом исследовании представлена небольшая справка.

Get the duration of dataset (Получаем продолжительность набора данных)

In [69]:
print("Starting date: ",historical_prices.iloc[0][0])
print("Ending date: ", historical_prices.iloc[-1][0])
print("Duration: ", historical_prices.iloc[-1][0]-historical_prices.iloc[0][0])

Starting date:  2011-03-18 00:00:00
Ending date:  2023-09-04 00:00:00
Duration:  4553 days 00:00:00


## Monthwise comparision between Stock actual, open and close price (Ежемесячное сравнение между ценой открытия и закрытия)

In [70]:
monthvise= historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))[['open','close']].mean().sort_values(by='close')
monthvise.head()

,open,close
date,,
September,3.134718,3.129435
October,3.157238,3.157908
August,3.180557,3.178014
July,3.224088,3.219453
November,3.228101,3.225891


In [71]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock actual open and close price')
fig.show()

## Monthwise High and Low stock price (Месячная максимальная и минимальная цена акций)

In [72]:
historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))['low'].min()

date
April        2.70
August       2.50
December     2.51
February     2.50
January      2.63
July         2.50
June         2.66
March        2.54
May          2.66
November     2.20
October      2.23
September    2.28
Name: low, dtype: float64

In [73]:
monthvise_high= historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))['high'].max()
monthvise_low= historical_prices.groupby(historical_prices['date'].dt.strftime('%B'))['low'].min()

In [74]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

## Momentum Strength
        Momentum is the velocity of price changes in a stock. It is used by investors to define if a stock can exhibit bullish trend, rising price, or bearish trend where the price is steadily falling.
        Моментум – это скорость изменения цены акции. Он используется инвесторами, чтобы определить, может ли акция демонстрировать бычий тренд, рост цены или медвежий тренд, когда цена неуклонно падает.

Если линия выше 0 - то тренд роста.
Если линия ниже 0 - то тренд падения.
Большой угол измения линии показывает силу тренда и локальную точку смены тренда.

In [75]:
df_cl = np.asarray(historical_prices['close'])
momentum = np.subtract(df_cl[10:], df_cl[:-10])
names = cycle(['Stock Close Price', 'momentum'])

fig = px.line(historical_prices, x=historical_prices.date[10:], y=[historical_prices['close'][10:],momentum],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.show()

## Arron indicator
        The Arron indicator is composed of two lines. 
        Considering a time range, an up line measures the number of periods since the highest price in the range, and a down line which measures the number of periods since the lowest price.
        Aroon indicates a bullish behavior when the Aroon up is above the Aroon down. 
        The opposite case indicates a bearish price behavior, and when the two lines cross each other can signal a trend changes.
        
        Что такое индикатор Aroon? Индикатор Aroon — это технический индикатор, который используется для определения трендовых изменений цены актива, а также силы этого тренда. 
        По сути, индикатор измеряет время между максимумами и время между минимумами за определенный период времени.

In [76]:
        period=7
        df_cl1 = np.asarray(historical_prices['close'])
        aroon_down=[(100/period)*
                 (period-np.argmax(df_cl1[t-period:t])) 
                 for t in range(period, len(df_cl1))]
        aroon_up=[(100/period)*
                   (period-np.argmin(df_cl1[t-period:t])) 
                   for t in range(period, len(df_cl1))]

names_1 = cycle(['Stock Close Price'])
names_2 = cycle(['aroon_up','aroon_down'])
fig = px.line(historical_prices, x=historical_prices.date[period:], y=[historical_prices['close'][period:]],
             labels={'date': 'Date','value':'Stock value'})
figs = px.line(historical_prices, x=historical_prices.date[period:], y=[aroon_up,aroon_down],
             labels={'date': 'Date','value':'Stock value'})

figs.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
figs.for_each_trace(lambda t:  t.update(name = next(names_2)))
figs.update_xaxes(showgrid=False)
figs.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names_1)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.show()
figs.show()

## Volatility
        Volatility is a measurement of the variation of prices over time.
    
        Method returns:
            -   Volatility (annualized standard deviation) numerical value, in percent
            -   Daily log return of the Close price

Волатильность – это измерение изменения цен во времени.
    
        Метод возвращает:
            - Числовое значение волатильности (стандартное отклонение в годовом исчислении), в процентах
            - Ежедневный лог возврата цены закрытия

In [77]:
def volatility_result(volatility_percent):
    volatility_percent = float(volatility_percent)
    if 12 <= volatility_percent <= 20:
        result = "the stock has normal volatility"
    elif volatility_percent < 12:
        result = "stock has low volatility"
    else:
        result = "stock has high volatility"
    print(result)
       
historical_prices['Log_returns'] = np.log(historical_prices['close']/historical_prices['close'].shift())
volatility_year = historical_prices['Log_returns'].std()*252**.5
volatility_percent = str(round(volatility_year, 4) * 100)
print(volatility_percent)
        
volatility_result(volatility_percent)

historical_prices.head()

25.39
stock has high volatility


,date,close,open,high,low,volume,Change %,Log_returns
3068,2011-03-18,4.05,4.10,4.10,4.04,103.47M,31.07%,NaN
3067,2011-03-21,4.09,4.12,4.12,4.06,84.48M,0.99%,0.009828
3066,2011-03-22,4.11,4.10,4.12,4.08,76.28M,0.49%,0.004878
3065,2011-03-23,4.08,4.13,4.13,4.08,71.41M,-0.73%,-0.007326
3064,2011-03-24,4.11,4.14,4.15,4.10,138.22M,0.74%,0.007326


In [125]:
# Найдем индекс строки с максимальным значением 'Log_returns'
max_log_returns_index = historical_prices['Log_returns'].idxmax()
min_log_returns_index = historical_prices['Log_returns'].idxmin()

# Получим дату по этому индексу
date_with_max_log_returns = historical_prices.loc[max_log_returns_index, 'date']
date_with_min_log_returns = historical_prices.loc[min_log_returns_index, 'date']

print("Дата с самым большим значением 'Log_returns':", date_with_max_log_returns)

print("Дата с самым маленьким значением 'Log_returns':", date_with_max_log_returns)

# Сортируем DataFrame по столбцу 'Log_returns' в порядке убывания, чтобы получить наибольшие значения первыми
sorted_df_descending = historical_prices.sort_values(by='Log_returns', ascending=False)

# Выбираем первые 10 строк с наибольшими значениями
top_max_log_returns = sorted_df_descending.head(25)

# Сортируем DataFrame по столбцу 'Log_returns' в порядке возрастания, чтобы получить наименьшие значения первыми
sorted_df_ascending = historical_prices.sort_values(by='Log_returns', ascending=True)

# Выбираем первые 10 строк с наименьшими значениями
top_min_log_returns = sorted_df_ascending.head(25)

# Выводим результаты
print("10 строк с наибольшими значениями 'Log_returns':")
print(top_max_log_returns)

print("\n10 строк с наименьшими значениями 'Log_returns':")
print(top_min_log_returns)

Дата с самым большим значением 'Log_returns': 2011-10-11 00:00:00
Дата с самым маленьким значением 'Log_returns': 2011-10-11 00:00:00
10 строк с наибольшими значениями 'Log_returns':
           date  close  open  high   low   volume Change %  Log_returns
2931 2011-10-11   2.99  2.97  3.07  2.91  613.03M   12.83%     0.120714
2894 2011-12-01   3.40  3.40  3.41  3.32  430.31M    8.97%     0.085942
2939 2011-09-27   2.87  2.75  2.88  2.71  246.86M    8.71%     0.083533
2922 2011-10-24   3.07  2.92  3.10  2.92  264.25M    8.10%     0.077874
850  2020-03-20   2.95  2.79  2.95  2.77  230.37M    7.66%     0.073847
780  2020-07-06   3.18  2.97  3.18  2.96  236.20M    7.43%     0.071692
616  2021-03-03   2.97  2.80  2.97  2.79  228.97M    6.83%     0.066111
2919 2011-10-27   3.55  3.45  3.59  3.43  438.91M    6.61%     0.063975
2933 2011-10-07   2.60  2.50  2.64  2.47  264.75M    6.56%     0.063513
2934 2011-10-06   2.44  2.46  2.47  2.37  372.74M    6.55%     0.063446
699  2020-11-02   2.79  2

In [126]:
fig = px.line(historical_prices, x=historical_prices.date, y=[historical_prices['Log_returns']],
             labels={'date': 'Date','value':'Volatility'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False,zeroline=True, zerolinewidth=2, zerolinecolor='Black')

fig.show()

Создание нормального распределения, для поиска уровня поддержки.  Уровень поддержки будет вяляться самым максимальынм значением Probability Density

In [148]:

# Создание нормального распределения
mean = top_max_log_returns['close'].mean()
std_dev = top_max_log_returns['close'].std()
x_top_max_log_returns = np.linspace(mean - 3*std_dev, mean + 3*std_dev, 100)
y_top_max_log_returns = (1/(std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)

# Создание графика
fig = px.line(x=x_top_max_log_returns, y=y_top_max_log_returns, labels={'x':'Close Value', 'y':'Probability Density'}, title='Normal Distribution of Close Values')
fig.show()


max_y_index_max_log_returns = np.argmax(y_top_max_log_returns)
x_at_max_y_max_log_returns = x_top_max_log_returns[max_y_index_max_log_returns]

print("Примерный уровень поддержки:", x_at_max_y_max_log_returns)

Примерный уровень поддержки: 3.8851774934550987


In [ ]:
Создание нормального распределения, для поиска уровня продажи. Уровень продажи будет вяляться самым максимальынм значением Probability Density

In [150]:
# Создание нормального распределения
mean_min_top_min_log_returns = top_min_log_returns['close'].mean()
std_dev_min_top_min_log_returns = top_min_log_returns['close'].std()
x_min_top_min_log_returns = np.linspace(mean_min_top_min_log_returns - 3*std_dev_min_top_min_log_returns, mean_min_top_min_log_returns + 3*std_dev_min_top_min_log_returns, 100)
y_min_top_min_log_returns = (1/(std_dev_min_top_min_log_returns * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean_min_top_min_log_returns) / std_dev_min_top_min_log_returns) ** 2)

# Создание графика
fig = px.line(x=x_min_top_min_log_returns, y=y_min_top_min_log_returns, labels={'x':'Close Value', 'y':'Probability Density'}, title='Normal Distribution of Close Values')
fig.show()

max_y_index_min_log_returns = np.argmax(y_min_top_min_log_returns)
x_at_max_y_min_log_returns  = x_min_top_min_log_returns[max_y_index_min_log_returns]

print("Примерный уровень сопротивления:", x_at_max_y_min_log_returns)

Примерный уровень сопротивления: 3.501696862074364


## Trend comparision between stock price, open price, close price, high price, low price

In [119]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(historical_prices, x=historical_prices.date, y=[historical_prices['open'], historical_prices['close'], 
                                          historical_prices['high'], historical_prices['low']],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()



# Close price prediction preparation and preprocessing (Подготовка к прогнозу цены и предварительная обработка)



Make separate dataframe with close price. (Подготовка отдельного кадра данных с ценой закрытия)

In [81]:
closedf = historical_prices[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (3067, 2)


Plotting stock close price

In [82]:
fig = px.line(closedf, x=closedf.date, y=closedf.close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.6)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Normalizing / scaling close value between 0 to 1 (Нормализация/масштабирование близкого значения от 0 до 1)

In [83]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(3067, 1)




## Split data for training and testing (Разделяем данные для обучения и тестирования)

Ratio for training and testing data is 65:35 (Соотношение данных для обучения и тестирования составляет 65:35)


In [84]:
training_size=int(len(closedf)*0.65)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (1993, 1)
test_data:  (1074, 1)


Create new dataset according to requirement of time-series prediction (Создание нового набора данных в соответствии с требованием прогнозирования временных рядов)

In [85]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [86]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (1977, 15)
y_train:  (1977,)
X_test:  (1058, 15)
y_test (1058,)


# Algorithms (Алгоритмы для предсказаний)

## Random Forest Regressor - RF

In [87]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [88]:
# Lets Do the prediction 

train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (1977, 1)
Test data prediction: (1058, 1)


In [89]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

### Evaluation metrices RMSE, MSE and MAE (Среднеквадратическая ошибка (RMSE), среднеквадратическая ошибка (MSE) и средняя абсолютная ошибка (MAE) )

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

Среднеквадратическая ошибка (RMSE), среднеквадратическая ошибка (MSE) и средняя абсолютная ошибка (MAE) — это стандартный способ измерения ошибки модели при прогнозировании количественных данных.

In [90]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  0.023817398629807342
Train data MSE:  0.0005672684774911486
Test data MAE:  0.017146788062721303
-------------------------------------------------------------------------------------
Test data RMSE:  0.07536441380170654
Test data MSE:  0.005679794867674856
Test data MAE:  0.04787136105860111




### Explained variance regression score (Объясненная оценка регрессии дисперсии)

The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

Показатель объясненной дисперсии объясняет дисперсию ошибок данного набора данных, и формула записывается следующим образом: Здесь и Var(y) — дисперсия ошибок прогнозирования и фактических значений соответственно. Крайне желательны оценки, близкие к 1,0, что указывает на лучшие квадраты стандартных отклонений ошибок.

In [91]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9970283458581711
Test data explained variance regression score: 0.9234003129644518


### R2 score for regression (Оценка R2 для регрессии)



R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model.

R-квадрат (R2) — это статистическая мера, представляющая долю дисперсии зависимой переменной, которая объясняется независимой переменной или переменными в регрессионной модели.

1 = Best
0 or < 0 = worse

In [92]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.997028343310346
Test data R2 score: 0.9149285538795139


### Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

Потеря регрессии Средняя потеря регрессии гамма-отклонения (MGD) и средняя потеря регрессии отклонения Пуассона (MPD)

In [93]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  4.663114875675578e-05
Test data MGD:  0.0008088892338969955
----------------------------------------------------------------------
Train data MPD:  0.00016072602550313634
Test data MPD:  0.002136102163050484


### Comparision between original stock close price vs predicted close price 
(Сравнение исходной цены закрытия акций с прогнозируемой ценой закрытия)

In [94]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (3067, 1)
Test predicted data:  (3067, 1)


Итог: цена прогназируемая повторяет оригинальные значения закрытия акции.

### Predicting next 15 days (Предсказание на следующие 15 дней)

In [95]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 60
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = regressor.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = regressor.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  60


### Plotting last 15 days and next predicted 15 days

In [96]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39
 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75]


In [97]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

names = cycle(['Last 15 days close price','Predicted next 15 days close price'])

min_stock_price_preducation = min(next_predicted_days_value)

max_stock_price_preducation = max(next_predicted_days_value)

print("Min stock price preducation", min_stock_price_preducation)
print("Max stock price preducation", max_stock_price_preducation)

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Min stock price preducation 2.51
Max stock price preducation 2.9638


### Plotting whole closing stock price with prediction (Построение всей цены акций на закрытие с прогнозированием)

In [98]:
rfdf=closedf.tolist()
rfdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
rfdf=scaler.inverse_transform(rfdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(rfdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.add_vrect(x0=closedf.size,x1=closedf.size)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [157]:
print("Results:")

print("Min stock price preducation", min_stock_price_preducation)
print("Max stock price preducation", max_stock_price_preducation)

print(f"Volatility: {volatility_percent} is the volatility of this stock for the year,\n i.e. the difference between the maximum and minimum price of the stock is {volatility_percent} percent.")

print(volatility_result(volatility_percent))

print("Information for forming support and resistance levels:")

print("Approximate resistance level (ideal selling price):", x_at_max_y_min_log_returns)

print("Approximate support level (ideal purchase price):", x_at_max_y_max_log_returns)

Results:
Min stock price preducation 2.51
Max stock price preducation 2.9638
Volatility: 25.39 is the volatility of this stock for the year,
 i.e. the difference between the maximum and minimum price of the stock is 25.39 percent.
stock has high volatility
None
Information for forming support and resistance levels:
Approximate resistance level (ideal selling price): 3.501696862074364
Approximate support level (ideal purchase price): 3.8851774934550987
